<a href="https://colab.research.google.com/github/Gaurav-phatkare/Amazon-Product-Reviews-Sentiment-Analysis/blob/main/Amazon_Product_review_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__) # version needed for Tensorflow_text library

2.13.0


In [2]:
# !pip install tensorflow_text

In [6]:
import tensorflow_text as text
import tensorflow_hub as hub

In [7]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)

In [8]:
import tensorflow as tf
assert isinstance(ds, tf.data.Dataset)
print(ds)

<_PrefetchDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), 'total_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'verified_purchase': TensorSpec(shape=(), dtype=tf.int64, name=None), 'vine': TensorSpec(shape=(), dtype

In [9]:
df = tfds.as_dataframe(ds)

In [10]:
df.head()

,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [11]:
df.shape

(104975, 15)

In [12]:
df["Sentiment"] = df["data/star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['short_review'] =df['data/review_body'].str.decode("utf-8")
df = df[["short_review", "Sentiment"]]

In [13]:
# Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

<ipython-input-13-d12344173841>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.tail(n).index,


In [14]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

In [ ]:
reviews

## Split the dataset into train and val

In [16]:
from sklearn.model_selection import train_test_split

train_sentances, val_sentances, train_labels, val_labels = train_test_split(df['short_review'].to_numpy(),
                                                                            df['Sentiment'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [17]:
# check the length
len(train_sentances), len(val_sentances)

(45000, 5000)

## Converting text into numbers

###Text Vectorizaion

In [18]:
import tensorflow as tf

from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None)

In [19]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentances])/len(train_sentances))

64

In [20]:
max_vocab_length = 20000
max_length = 64

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [21]:
text_vectorizer.adapt(train_sentances)

In [22]:
# Create sample sentence and tokenize it
sample_sentence = "The product was too good than my expectations"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[  2,  36,  19, 108,  30,  62,  11, 982,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [23]:
#Choose a random sentence from the training dataset and tokenize it
import random
random_sentance = random.choice(train_sentances)
print(f"Original text:\n{random_sentance}\
      \n\nVectorized version:")
text_vectorizer([random_sentance])

Original text:
Speaker looks nice, but I get nothing but static when connected to Bluetooth. Returned the speaker, hopefully the replacement works better.      

Vectorized version:


<tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[ 103,  160,   95,   20,    3,   55,  346,   20,  692,   38,  552,
           6,  185,  615,    2,  103, 1406,    2,  371,   56,  104,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0]])>

In [24]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

In [25]:
words_in_vocab[:5]

['', '[UNK]', 'the', 'i', 'it']

In [26]:
words_in_vocab[-5:]

['idorks', 'identico', 'identically', 'ideapad', 'ideacentre']

In [27]:
len(words_in_vocab)

20000

## create embedding using an embedding layer

In [28]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                      embeddings_initializer = 'uniform',
                                      input_length = max_length,
                                      name = 'embedding_1')

In [29]:
# Get a random sentence from training set
random_sentance = random.choice(train_sentances)
print(f"Original text:\n{random_sentance}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentance]))
sample_embed

Original text:
simple to use,rough,can take a hit and sold at a good price<br /><br />what else can i say. can't go wrong      

Embedded version:


<tf.Tensor: shape=(1, 64, 128), dtype=float32, numpy=
array([[[-0.01789083,  0.03433895, -0.04231099, ..., -0.0137537 ,
         -0.01838279, -0.02459972],
        [-0.01187982,  0.00782765, -0.04283259, ..., -0.03897598,
          0.03717067,  0.04288466],
        [-0.01193165, -0.00562358, -0.04208972, ..., -0.02028378,
         -0.01954993, -0.0256142 ],
        ...,
        [ 0.03781916,  0.01543095, -0.03002666, ..., -0.02753578,
          0.04331124,  0.04234935],
        [ 0.03781916,  0.01543095, -0.03002666, ..., -0.02753578,
          0.04331124,  0.04234935],
        [ 0.03781916,  0.01543095, -0.03002666, ..., -0.02753578,
          0.04331124,  0.04234935]]], dtype=float32)>

# Model 0: Baseline model

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',MultinomialNB())
])

# fit the pipeline to the training data

model_0.fit(train_sentances, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [31]:
baseline_score = model_0.score(val_sentances, val_labels)

In [32]:
baseline_score

0.7978

In [33]:
baseline_preds = model_0.predict(val_sentances)
baseline_preds[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [34]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [35]:
baseline_results = calculate_results(val_labels, baseline_preds)

In [36]:
baseline_results

{'accuracy': 79.78,
 'precision': 0.8338266657514186,
 'recall': 0.7978,
 'f1': 0.7308971860355469}

# model 1: A simple dense model

In [37]:
# Build model with the functional API
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs, name = 'model_1_dense')

In [38]:
# compile a model

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 64, 128)           2560000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2560129 (9.77 MB)
Trainable params: 256

In [40]:
# fit the model

model_1_history = model_1.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 64s 44ms/step - loss: 0.3615 - accuracy: 0.8398 - val_loss: 0.2729 - val_accuracy: 0.8826
Epoch 2/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.2501 - accuracy: 0.8939 - val_loss: 0.2531 - val_accuracy: 0.8886
Epoch 3/5
1407/1407 [==============================] - 12s 8ms/step - loss: 0.2084 - accuracy: 0.9128 - val_loss: 0.2505 - val_accuracy: 0.8944
Epoch 4/5
1407/1407 [==============================] - 12s 9ms/step - loss: 0.1744 - accuracy: 0.9294 - val_loss: 0.2532 - val_accuracy: 0.8934
Epoch 5/5
1407/1407 [==============================] - 12s 9ms/step - loss: 0.1455 - accuracy: 0.9454 - val_loss: 0.2618 - val_accuracy: 0.8954


In [41]:
model_1.evaluate(val_sentances, val_labels)

157/157 [==============================] - 1s 5ms/step - loss: 0.2618 - accuracy: 0.8954


[0.2617794871330261, 0.8953999876976013]

In [42]:
# model 1 prediction probability

model_1_pred_prob = model_1.predict(val_sentances)

157/157 [==============================] - 1s 3ms/step


In [43]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_prob))

In [44]:
model_1_preds

<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [45]:
# Calculate model_1 metrics
model_1_results = calculate_results(val_labels,
                                    model_1_preds)
model_1_results

{'accuracy': 89.53999999999999,
 'precision': 0.8923020491556636,
 'recall': 0.8954,
 'f1': 0.8927199681948997}

# model 2: LSTM

In [46]:
from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 256,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_2_model')

# build a model
inputs = layers.Input(shape =(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x  = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

(None, 64, 256)
(None, 64)


In [47]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [48]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_2_model (Embeddi  (None, 64, 256)           5120000   
 ng)                                                             
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5202241 (19.84 MB)
Trainable params: 520

In [49]:
# Fit model
model_2_history = model_2.fit(train_sentances,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 72s 49ms/step - loss: 0.4903 - accuracy: 0.7779 - val_loss: 0.5328 - val_accuracy: 0.7574
Epoch 2/5
1407/1407 [==============================] - 20s 15ms/step - loss: 0.4600 - accuracy: 0.8078 - val_loss: 0.4911 - val_accuracy: 0.8160
Epoch 3/5
1407/1407 [==============================] - 20s 14ms/step - loss: 0.3262 - accuracy: 0.8558 - val_loss: 0.2762 - val_accuracy: 0.8830
Epoch 4/5
1407/1407 [==============================] - 21s 15ms/step - loss: 0.2074 - accuracy: 0.9165 - val_loss: 0.2653 - val_accuracy: 0.8886
Epoch 5/5
1407/1407 [==============================] - 17s 12ms/step - loss: 0.1542 - accuracy: 0.9408 - val_loss: 0.3024 - val_accuracy: 0.8860


In [50]:
# make some predictions

model_2_pred_probs = model_2.predict(val_sentances)

157/157 [==============================] - 1s 6ms/step


In [51]:
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [52]:
# Calculate LSTM model results
model_2_results = calculate_results(val_labels,
                                    model_2_pred)
model_2_results

{'accuracy': 88.6,
 'precision': 0.8885821377468064,
 'recall': 0.886,
 'f1': 0.8871016594196469}

# Model 3: GRU

In [53]:
# import tensorflow as tf
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_3_mode')


# build a Rnn with GRU cell

inputs  = layers.Input(shape=(1,), dtype = 'string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64, dropout=0.25)(x)
# x = layers.Dense(64, activation = 'relu')(x)
x = layers.Dense(64, activation = 'relu')(x)
# model.add(keras.layers.Dropout(0.4))
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_GRU')

In [54]:
# compile the Model

model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [55]:
# summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_3_mode (Embeddin  (None, 64, 128)           2560000   
 g)                                                              
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                

In [56]:


# fit the model

model_3_history = model_3.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 59s 39ms/step - loss: 0.4170 - accuracy: 0.8187 - val_loss: 0.2489 - val_accuracy: 0.8946
Epoch 2/5
1407/1407 [==============================] - 19s 14ms/step - loss: 0.2284 - accuracy: 0.9058 - val_loss: 0.2444 - val_accuracy: 0.8972
Epoch 3/5
1407/1407 [==============================] - 18s 13ms/step - loss: 0.1781 - accuracy: 0.9298 - val_loss: 0.2558 - val_accuracy: 0.8922
Epoch 4/5
1407/1407 [==============================] - 17s 12ms/step - loss: 0.1345 - accuracy: 0.9480 - val_loss: 0.3005 - val_accuracy: 0.8910
Epoch 5/5
1407/1407 [==============================] - 18s 13ms/step - loss: 0.1005 - accuracy: 0.9629 - val_loss: 0.3639 - val_accuracy: 0.8840


In [57]:
# Epoch 1/5
# 1407/1407 [==============================] - 48s 32ms/step - loss: 0.4128 - accuracy: 0.8209 - val_loss: 0.2496 - val_accuracy: 0.8944
# Epoch 2/5
# 1407/1407 [==============================] - 15s 11ms/step - loss: 0.2227 - accuracy: 0.9084 - val_loss: 0.2412 - val_accuracy: 0.8984
# Epoch 3/5
# 1407/1407 [==============================] - 15s 11ms/step - loss: 0.1578 - accuracy: 0.9385 - val_loss: 0.2747 - val_accuracy: 0.8898
# Epoch 4/5
# 1407/1407 [==============================] - 13s 10ms/step - loss: 0.1037 - accuracy: 0.9613 - val_loss: 0.3157 - val_accuracy: 0.8910
# Epoch 5/5
# 1407/1407 [==============================] - 14s 10ms/step - loss: 0.0687 - accuracy: 0.9758 - val_loss: 0.4189 - val_accuracy: 0.8876

In [58]:
# evaluate the model
model_3.evaluate(val_sentances)

157/157 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [59]:
# Make predictions on the validation data

model_3_pred_probs = model_3.predict(val_sentances)
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred

157/157 [==============================] - 1s 6ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [60]:
# Calculate LSTM model results
model_3_results = calculate_results(val_labels,
                                    model_3_pred)
model_3_results

{'accuracy': 88.4,
 'precision': 0.8841430942492651,
 'recall': 0.884,
 'f1': 0.8840708353079993}

# Model 4 : Bidirectional RNN Model

In [61]:
from tensorflow.keras import layers

model_4_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_4_model')

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [62]:
# compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [63]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_4_model (Embeddi  (None, 64, 128)           2560000   
 ng)                                                             
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                             

In [64]:
# Fit the model
model_4_history = model_4.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 64s 41ms/step - loss: 0.3108 - accuracy: 0.8696 - val_loss: 0.2567 - val_accuracy: 0.8978
Epoch 2/5
1407/1407 [==============================] - 25s 18ms/step - loss: 0.2173 - accuracy: 0.9127 - val_loss: 0.2478 - val_accuracy: 0.8996
Epoch 3/5
1407/1407 [==============================] - 22s 16ms/step - loss: 0.1652 - accuracy: 0.9346 - val_loss: 0.2945 - val_accuracy: 0.8824
Epoch 4/5
1407/1407 [==============================] - 21s 15ms/step - loss: 0.1166 - accuracy: 0.9561 - val_loss: 0.3262 - val_accuracy: 0.8832
Epoch 5/5
1407/1407 [==============================] - 25s 18ms/step - loss: 0.0816 - accuracy: 0.9705 - val_loss: 0.3694 - val_accuracy: 0.8846


In [65]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentances)
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred

157/157 [==============================] - 2s 5ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [66]:
# Calculate model 4 results
model_4_results = calculate_results(val_labels,
                                    model_4_pred)
model_4_results

{'accuracy': 88.46000000000001,
 'precision': 0.8847065770750987,
 'recall': 0.8846,
 'f1': 0.8846528873998796}

# Model 5: Conv1D

In [67]:
# test out the embedding , 1D convolutional and max pooling

embedding_test = embedding(text_vectorizer(["Packaging of product was too bad"]))

conv_1d = layers.Conv1D(filters = 32, kernel_size = 5, activation = 'relu')
conv_1d_output = conv_1d(embedding_test)

max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 64, 128]), TensorShape([1, 60, 32]), TensorShape([1, 32]))

In [68]:
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 64, 128), dtype=float32, numpy=
 array([[[-0.0154944 , -0.03652706, -0.04778898, ..., -0.09199777,
          -0.09814937, -0.09431557],
         [-0.0719146 , -0.05856934, -0.08947874, ..., -0.13667926,
          -0.12404729, -0.13513768],
         [-0.036557  , -0.05766684, -0.03341823, ..., -0.22974378,
          -0.1923285 , -0.18168348],
         ...,
         [-0.06031284, -0.07602931, -0.0911251 , ..., -0.37208846,
          -0.18996136, -0.15337022],
         [-0.06031284, -0.07602931, -0.0911251 , ..., -0.37208846,
          -0.18996136, -0.15337022],
         [-0.06031284, -0.07602931, -0.0911251 , ..., -0.37208846,
          -0.18996136, -0.15337022]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 60, 32), dtype=float32, numpy=
 array([[[0.        , 0.        , 0.        , ..., 0.13052216,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.07344185,
          0.05924364, 0.        ],
         [0.14001817, 0.        , 0.20

In [69]:
# build a model
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_5')


from tensorflow.keras import layers

inputs = layers.Input(shape =(1, ), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_5_embedding(x)

x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation = 'relu', name = 'extraLayer')(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [70]:
# compile the model

model_5.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 64, 128)           2560000   
                                                                 
 conv1d_1 (Conv1D)           (None, 60, 32)            20512     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_5 (Dense)             (None, 1)              

In [71]:
# Fit the model
model_5_history = model_5.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 51s 34ms/step - loss: 0.3170 - accuracy: 0.8608 - val_loss: 0.2565 - val_accuracy: 0.8954
Epoch 2/5
1407/1407 [==============================] - 15s 11ms/step - loss: 0.2066 - accuracy: 0.9156 - val_loss: 0.2559 - val_accuracy: 0.8922
Epoch 3/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.1342 - accuracy: 0.9507 - val_loss: 0.2805 - val_accuracy: 0.8894
Epoch 4/5
1407/1407 [==============================] - 12s 9ms/step - loss: 0.0771 - accuracy: 0.9756 - val_loss: 0.3283 - val_accuracy: 0.8856
Epoch 5/5
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0407 - accuracy: 0.9892 - val_loss: 0.3649 - val_accuracy: 0.8828


In [72]:
# # Fit the model
# model_5_history = model_5.fit(train_sentances,
#                               train_labels,
#                               epochs = 5,
#                               validation_data = (val_sentances, val_labels))

In [73]:
# Make Predictions
model_5_pred_prob = model_5.predict(val_sentances)
model_5_pred = tf.squeeze(tf.round(model_5_pred_prob))
model_5_pred

157/157 [==============================] - 1s 3ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 0., 1.], dtype=float32)>

In [74]:
# Calculate 5th model results
model_5_results = calculate_results(val_labels,
                                    model_5_pred)
model_5_results

{'accuracy': 88.28,
 'precision': 0.883165242420133,
 'recall': 0.8828,
 'f1': 0.8829781968266343}

# Model 6: pretrain models from Tensorflow Hub

In [75]:
import tensorflow_hub as hub

embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')


In [76]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [77]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [78]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentances,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 52s 34ms/step - loss: 0.2878 - accuracy: 0.8739 - val_loss: 0.2575 - val_accuracy: 0.8916
Epoch 2/5
1407/1407 [==============================] - 46s 33ms/step - loss: 0.2557 - accuracy: 0.8896 - val_loss: 0.2529 - val_accuracy: 0.8940
Epoch 3/5
1407/1407 [==============================] - 47s 33ms/step - loss: 0.2493 - accuracy: 0.8924 - val_loss: 0.2474 - val_accuracy: 0.8960
Epoch 4/5
1407/1407 [==============================] - 52s 37ms/step - loss: 0.2433 - accuracy: 0.8956 - val_loss: 0.2461 - val_accuracy: 0.8996
Epoch 5/5
1407/1407 [==============================] - 47s 33ms/step - loss: 0.2376 - accuracy: 0.8982 - val_loss: 0.2410 - val_accuracy: 0.8962


In [79]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentances)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds

157/157 [==============================] - 4s 23ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [80]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 89.62,
 'precision': 0.8937475750287,
 'recall': 0.8962,
 'f1': 0.8944811486131653}

# Model 7: 10 percent data on model6

In [81]:
import numpy as np
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentances),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
                                                                                                                            random_state=42)

In [82]:
len(train_sentences_10_percent),len(train_labels_10_percent)

(4500, 4500)

In [83]:
len(train_sentences_90_percent)

40500

In [84]:
import pandas as pd
pd.Series(train_labels_10_percent).value_counts()

1    3419
0    1081
dtype: int64

In [85]:

model_7 = tf.keras.models.clone_model(model_6)

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [86]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentances, val_labels),)

Epoch 1/5
141/141 [==============================] - 12s 64ms/step - loss: 0.4550 - accuracy: 0.7911 - val_loss: 0.3298 - val_accuracy: 0.8538
Epoch 2/5
141/141 [==============================] - 10s 73ms/step - loss: 0.2916 - accuracy: 0.8804 - val_loss: 0.2773 - val_accuracy: 0.8814
Epoch 3/5
141/141 [==============================] - 14s 97ms/step - loss: 0.2619 - accuracy: 0.8940 - val_loss: 0.2724 - val_accuracy: 0.8794
Epoch 4/5
141/141 [==============================] - 7s 49ms/step - loss: 0.2521 - accuracy: 0.8984 - val_loss: 0.2703 - val_accuracy: 0.8820
Epoch 5/5
141/141 [==============================] - 11s 78ms/step - loss: 0.2455 - accuracy: 0.8993 - val_loss: 0.2684 - val_accuracy: 0.8816


In [87]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentances)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds

157/157 [==============================] - 4s 24ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [88]:
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

{'accuracy': 88.16000000000001,
 'precision': 0.87781968440438,
 'recall': 0.8816,
 'f1': 0.8787209385276256}

# Model 8:BERT Model

In [89]:
# !pip install tensorflow_text

In [90]:
# import tensorflow_hub as hub
# # import tensorflow_text as text

# https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [91]:
def sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

In [92]:
# Bert layer
inputs = tf.keras.layers.Input(shape =(), dtype = tf.string,name = 'text')
preprocess_layer = bert_preprocess(inputs)

outputs = bert_encoder(preprocess_layer)

x = layers.Dropout(0.2, name = "dropout_1")(outputs['pooled_output'])
x = layers.Dense(1, activation='sigmoid', name = 'output')(x)


BERT_model = tf.keras.Model(inputs=[inputs],outputs = [x])

In [93]:
BERT_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [94]:
# compile a model
BERT_model.compile(loss = 'binary_crossentropy',
                   optimizer = tf.keras.optimizers.Adam(),
                   metrics=['accuracy'])

In [95]:
import numpy as np
train_sentences_80_percent, train_sentences_20_percent, train_labels_80_percent, train_labels_20_percent = train_test_split(np.array(train_sentances),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.2,
                                                                                                                            random_state=42)

In [96]:
len(train_sentences_20_percent)

9000

In [97]:
#fit the model
Bert_model_history = BERT_model.fit(train_sentences_20_percent,
                              train_labels_20_percent,
                              epochs=16,
                              validation_data=(val_sentances, val_labels))

Epoch 1/16
282/282 [==============================] - 163s 538ms/step - loss: 0.5309 - accuracy: 0.7631 - val_loss: 0.4763 - val_accuracy: 0.7706
Epoch 2/16
282/282 [==============================] - 183s 650ms/step - loss: 0.4834 - accuracy: 0.7746 - val_loss: 0.4477 - val_accuracy: 0.7742
Epoch 3/16
282/282 [==============================] - 182s 647ms/step - loss: 0.4582 - accuracy: 0.7819 - val_loss: 0.4254 - val_accuracy: 0.7858
Epoch 4/16
282/282 [==============================] - 182s 647ms/step - loss: 0.4418 - accuracy: 0.7910 - val_loss: 0.4066 - val_accuracy: 0.8066
Epoch 5/16
282/282 [==============================] - 182s 646ms/step - loss: 0.4283 - accuracy: 0.7968 - val_loss: 0.3941 - val_accuracy: 0.8154
Epoch 6/16
282/282 [==============================] - 182s 647ms/step - loss: 0.4263 - accuracy: 0.7983 - val_loss: 0.4303 - val_accuracy: 0.7768
Epoch 7/16
282/282 [==============================] - 156s 554ms/step - loss: 0.4152 - accuracy: 0.8038 - val_loss: 0.3786 -

In [98]:
BERT_model_pred_probs = BERT_model.predict(val_sentances)
BERT_model_pred = tf.squeeze(tf.round(BERT_model_pred_probs))
BERT_model_pred

157/157 [==============================] - 56s 348ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [99]:
BERT_results = calculate_results(val_labels,BERT_model_pred)

# Compare the all results

In [100]:
# Combine model results into a DataFrame
all_results_models = pd.DataFrame({"0_baseline": baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results,
                                  "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results,
                                  "5_conv1d": model_5_results,
                                  "6_tf_hub_sentence_encoder": model_6_results,
                                  "7_tf_hub_10_percent_data": model_7_results,
                                   "BERT_model":BERT_results})
all_results_models = all_results_models.transpose()
all_results_models

,accuracy,precision,recall,f1
0_baseline,79.78,0.833827,0.7978,0.730897
1_simple_dense,89.54,0.892302,0.8954,0.892720
2_lstm,88.60,0.888582,0.8860,0.887102
3_gru,88.40,0.884143,0.8840,0.884071
4_bidirectional,88.46,0.884707,0.8846,0.884653
5_conv1d,88.28,0.883165,0.8828,0.882978
6_tf_hub_sentence_encoder,89.62,0.893748,0.8962,0.894481
7_tf_hub_10_percent_data,88.16,0.877820,0.8816,0.878721
BERT_model,84.68,0.842094,0.8468,0.828810


In [144]:
pred_prob = BERT_model.predict(["quality is bad"])
pred_prob[0][0]

1/1 [==============================] - 0s 79ms/step


0.69756836

In [151]:
# def predict_on_sentence(model, sentence):
#   """
#   Uses model to make a prediction on sentence.

#   Returns the sentence, the predicted label and the prediction probability.
#   """
#   pred_prob = model.predict([sentence])
#   pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
#   print(f"Pred: {pred_label}", "Positive" if pred_label > 0 else "Negative", f"Prob: {pred_prob[0][0]}")
#   print(f"Text:\n{sentence}")
def predict_on_sentence(model, sentence):
  pred_prob = model.predict([sentence], verbose = 0)
  pred_label = tf.squeeze(tf.round(pred_prob)).numpy()

  if pred_label > 0 :
    return ["Positive",f"{pred_prob[0][0]}"]
  else:
    return ["Negative",f'{pred_prob[0][0]}']

In [152]:
# predict_on_sentence(model_2, "product was not bad ")

In [155]:
text_sentance = "Product quality is not bad!!!"

In [156]:
results_all = {
    "model_1_Simple_Dense ":predict_on_sentence(model_1, text_sentance),
    "model_2_LSTM":predict_on_sentence(model_2, text_sentance),
    "model_3_GRU":predict_on_sentence(model_3, text_sentance),
    "model_4_Bidirectional":predict_on_sentence(model_4, text_sentance),
    "model_5_Conv1D":predict_on_sentence(model_5, text_sentance),
    "model_6_pretrained":predict_on_sentence(model_6, text_sentance),
    "model_7_pretrained_10%":predict_on_sentence(model_7, text_sentance),
    "model_8_Bert":predict_on_sentence(BERT_model, text_sentance),
}

column = ['Prediction','Probability']
pd.DataFrame.from_dict(results_all, orient='index', columns=column)

,Prediction,Probability
model_1_Simple_Dense,Negative,0.34135544300079346
model_2_LSTM,Negative,0.19569820165634155
model_3_GRU,Negative,0.18696321547031403
model_4_Bidirectional,Positive,0.5759806632995605
model_5_Conv1D,Negative,0.2628621459007263
model_6_pretrained,Positive,0.593865156173706
model_7_pretrained_10%,Negative,0.4179008901119232
model_8_Bert,Positive,0.7517614960670471
